In [6]:
# standard
from IPython import embed
import pandas as pd
import numpy as np

# frameworks
from frameworks.seq2seq_keras.models import AttentionSeq2Seq
from gensim.models import Word2Vec

# custom
from data_utils import get_train_data
from word2vec import get_word_embedding
from vocab import get_vocab

Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.424 seconds.
Prefix dict has been built succesfully.


# Global Variables

In [7]:
_BATCH_SIZE = 64
_VOCAB_SIZE = 6000
_WORD_DIM = 128
_MODEL_DEPTH = 4

_INPUT_LENGTH = 25
_OUTPUT_LENGTH = 10

# Model

In [3]:
model = AttentionSeq2Seq(input_length=_INPUT_LENGTH, 
                         input_dim=_WORD_DIM, 
                         hidden_dim=_WORD_DIM, 
                         output_length=_OUTPUT_LENGTH, 
                         output_dim=_WORD_DIM, 
                         depth=_MODEL_DEPTH)
model.compile(loss='mse', optimizer='rmsprop')

# Data

In [8]:
embedding = get_word_embedding(_WORD_DIM)

In [7]:
train_data = get_train_data()
_, ch2int = get_vocab()

In [8]:
len(train_data)

39956

In [9]:
def pad_to(lst, length, value):
    for i in range(len(lst), length):
        lst.append(value)
    
    return lst

def clean_train_data(train_data):
    X_train = []
    Y_train = []
    for idx in xrange(len(train_data)):
        line_number = idx % 4
        
        keyword = train_data[idx]['keyword']
        current_sentence = train_data[idx]['sentence']
        previous_sentences = ''.join([train_data[idx - i]['sentence'] for i in range(line_number, 0, -1)])
        
        X_entry = pad_to([[ch2int[ch]] for ch in (keyword + previous_sentences)], 25, [_VOCAB_SIZE - 1])
        Y_entry = pad_to([[ch2int[ch]] for ch in current_sentence], 10, [_VOCAB_SIZE - 1])
        
        X_train.append(X_entry)
        Y_train.append(Y_entry)
        
    return X_train, Y_train

In [10]:
X_train, Y_train = clean_train_data(train_data)

In [13]:
X_train_embedded = [map(lambda x: embedding[x[0]], sample) for sample in X_train]

In [14]:
Y_train_embedded = [map(lambda x: embedding[x[0]], sample) for sample in Y_train]

# Training

In [15]:
model.fit(X_train_embedded, Y_train_embedded, epochs=1, verbose=1)

Epoch 1/1
39956/39956 [==============================] - 421s - loss: 0.4278   


# Generation

In [16]:
kw = u'山水'

In [17]:
kw_pad = [pad_to([[ch2int[ch]] for ch in kw], 25, [_VOCAB_SIZE - 1])]

In [18]:
kw_embed = [map(lambda x: embedding[x[0]], sample) for sample in kw_pad]

In [19]:
kw_embed_array = np.array(kw_embed)

In [20]:
pred = model.predict(kw_embed_array)
pred

array([[[-0.08963315,  0.13422257, -0.24570994, ..., -0.14995289,
          0.17542832,  0.13692029],
        [-0.03718845,  0.30552149, -0.17106384, ..., -0.05145008,
          0.19576812,  0.19635646],
        [-0.08512392,  0.32059655, -0.13894799, ..., -0.04305276,
          0.11034762,  0.05403135],
        ..., 
        [ 0.82171226,  0.54636025,  0.8892892 , ..., -0.24688512,
         -0.63101834,  0.0330935 ],
        [ 0.84401846,  0.55392796,  0.92472196, ..., -0.2636658 ,
         -0.64143229,  0.04308706],
        [ 0.84558034,  0.54933226,  0.92686731, ..., -0.2670286 ,
         -0.64220965,  0.04647564]]], dtype=float32)

In [21]:
w2v_model = Word2Vec.load('data/word2vec.model')

In [22]:
result = []
for i in range(len(pred[0])):
    result.append(w2v_model.most_similar(positive=[pred[0][i]], topn=1))

In [4]:
for r in result:
    print r[0][0]

NameError: name 'result' is not defined

In [10]:
embedding[5999]

array([ 0.8498039 ,  0.57556695,  0.92304476,  0.17803186,  0.04769286,
        0.99453824, -0.82562774,  0.53770008,  0.73325625,  0.63910316,
        0.39900191,  0.30091339, -0.03310214,  0.44781277, -0.71808411,
       -0.07415888, -0.35380776, -0.38131324,  0.61963274, -0.91428909,
       -0.27381443, -0.4955787 ,  0.20827111,  0.8872208 , -0.34586555,
       -0.40137233, -0.85387734, -0.21692254, -0.04420312,  0.98681227,
       -0.52111557, -0.56008486, -0.25052312, -0.28824903, -0.11583056,
       -0.46158419,  0.82141598,  0.99022346,  0.00357126,  0.19400877,
       -0.83994324,  0.14695139, -0.77157749, -0.91833494,  0.18459308,
       -0.35598933, -0.45214338, -0.84804342,  0.71834489, -0.88759671,
        0.2754031 , -0.45306053,  0.40938456, -0.9046748 , -0.15501666,
       -0.11718657, -0.8087438 ,  0.05712781, -0.74951334,  0.54363642,
       -0.15289607, -0.97428664,  0.65542609, -0.61612186, -0.9322828 ,
        0.44299893,  0.4208495 , -0.85443294, -0.7864411 ,  0.60